In [1]:
# --- Setup: Imports and Path Adjustments ---
import pandas as pd
import sys
import os

# Adjust sys.path to allow imports from src/
current_notebook_dir = os.getcwd()  # Use current working directory for Jupyter notebooks
project_root_dir = os.path.join(current_notebook_dir, '..')  # Go up to 'solar-challenge-week1/'
sys.path.insert(0, project_root_dir)

# Import functions from your src modules
import scripts.data_cleaning as dc
import scripts.data_profiling as dp
import scripts.eda_plots as ep

# Define country name for consistent naming
COUNTRY = "Sierraleone"
DATA_FILE = "sierraleone-bumbuna.csv"

# **1. Load Raw Data**

We begin by loading the raw data for Benin. The dataset is expected to be in the `data/` directory. If the file is missing, an error will be raised.

In [2]:
# --- 1. Load Raw Data ---
print(f"--- Loading Raw Data for {COUNTRY} ---")
data_path = os.path.join(project_root_dir, 'data', DATA_FILE)

try:
    df_raw = pd.read_csv(data_path)
    print(f"Successfully loaded {DATA_FILE}.")
except FileNotFoundError:
    print(f"Error: Raw data file not found at {data_path}. Please check the path and file existence.")
    sys.exit("Data file not found.")

print("\n**Raw Data Head:**")
display(df_raw.head())
print(f"\n**Raw Data Shape:** {df_raw.shape}")

--- Loading Raw Data for Sierraleone ---
Successfully loaded sierraleone-bumbuna.csv.

**Raw Data Head:**


,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-10-30 00:01,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.1,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
1,2021-10-30 00:02,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
2,2021-10-30 00:03,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.2,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN
3,2021-10-30 00:04,-0.7,0.0,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.1,22.3,22.6,NaN
4,2021-10-30 00:05,-0.7,-0.1,-0.8,0.0,0.0,21.9,99.3,0.0,0.0,0.0,0.0,0.0,1002,0,0.0,22.3,22.6,NaN



**Raw Data Shape:** (525600, 19)


# **2. Initial Data Profiling**

Before cleaning, we perform an initial data profiling to understand the raw state of the dataset. This includes summary statistics and a missing value report.

In [3]:
# --- 2. Initial Data Profiling (Raw Data) ---
print("\n--- Initial Data Profiling (Raw Data) ---")
display(dp.get_summary_statistics(df_raw.select_dtypes(include=['number'])))
dp.print_missing_value_report(df_raw, threshold=5)  # List columns with >5% nulls


--- Initial Data Profiling (Raw Data) ---

--- Summary Statistics ---


,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
count,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,0.0
mean,201.957515,116.376337,113.720571,206.643095,198.114691,26.319394,79.448857,1.146113,1.691606,0.363823,133.044668,7.172220,999.876469,0.000967,0.004806,32.504263,32.593091,NaN
std,298.495150,218.652659,158.946032,300.896893,288.889073,4.398605,20.520775,1.239248,1.617053,0.295000,114.284792,7.535093,2.104419,0.031074,0.047556,12.434899,12.009161,NaN
min,-19.500000,-7.800000,-17.900000,0.000000,0.000000,12.300000,9.900000,0.000000,0.000000,0.000000,0.000000,0.000000,993.000000,0.000000,0.000000,10.700000,11.100000,NaN
25%,-2.800000,-0.300000,-3.800000,0.000000,0.000000,23.100000,68.700000,0.000000,0.000000,0.000000,0.000000,0.000000,999.000000,0.000000,0.000000,23.500000,23.800000,NaN
50%,0.300000,-0.100000,-0.100000,3.600000,3.400000,25.300000,85.400000,0.800000,1.600000,0.400000,161.500000,6.200000,1000.000000,0.000000,0.000000,26.600000,26.900000,NaN
75%,362.400000,107.000000,224.700000,359.500000,345.400000,29.400000,96.700000,2.000000,2.600000,0.600000,234.100000,12.000000,1001.000000,0.000000,0.000000,40.900000,41.300000,NaN
max,1499.000000,946.000000,892.000000,1507.000000,1473.000000,39.900000,100.000000,19.200000,23.900000,4.100000,360.000000,98.400000,1006.000000,1.000000,2.400000,72.800000,70.400000,NaN



--- Missing Value Report ---
Missing values by column:
          Missing Count  Missing Percentage
Comments         525600               100.0

Columns with more than 5% missing values:
          Missing Count  Missing Percentage
Comments         525600               100.0


# **3. Clean Data**

The raw data is cleaned using the `clean_data` function from `src/data_cleaning.py`. This process includes handling missing values, removing outliers, and ensuring data integrity.

In [4]:
# --- 3. Clean Data ---
print(f"\n--- Cleaning Data for {COUNTRY} using src/data_cleaning.py ---")
df_cleaned = dc.clean_data(df_raw.copy(), country_name=COUNTRY)

print("\n**Cleaned Data Head:**")
display(df_cleaned.head())
print(f"\n**Cleaned Data Shape:** {df_cleaned.shape}")


--- Cleaning Data for Sierraleone using src/data_cleaning.py ---

--- Cleaning Data for Sierraleone ---

  Enforcing Physical Constraints (Setting impossible values to NaN)...
    Corrected 261135 negative values in 'GHI'.
    Corrected 266352 negative values in 'DNI'.
    Corrected 263128 negative values in 'DHI'.
  Total physically impossible values corrected: 790615

  Dropped 286728 rows due to missing values in critical columns: GHI, DNI, DHI, ModA, ModB.

  Imputing remaining numerical missing values with median...


c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\.env\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\notebooks\..\scripts\data_cleaning.py:123: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Comments'].fillna('No Comment', inplace=True)
c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\notebooks\..\scripts\

  No further numerical missing values required imputation.
  Filled 238872 missing 'Comments' with 'No Comment'.

  Detecting potential statistical outliers (Z-score > 3) for reporting...
  Potential statistical outliers detected (Z-score > 3):
    - GHI: 81 outliers
    - DHI: 415 outliers
    - ModA: 33 outliers
    - ModB: 51 outliers
    - Tamb: 339 outliers
    - WS: 915 outliers
    - WSgust: 1140 outliers
    - Precipitation: 1638 outliers
  Note: These values are flagged for awareness but not automatically removed or capped by this function.

--- Cleaning Summary for Sierraleone ---
  Initial rows: 525600
  Final rows after cleaning: 238872
  Total rows removed/adjusted: 286728

  Missing values after cleaning:
Series([], dtype: int64)

Cleaned data saved to: c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\scripts\..\data\sierraleone_clean.csv

**Cleaned Data Head:**


,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
Timestamp,,,,,,,,,,,,,,,,,,
2021-10-30 07:13:00,17.7,0.0,17.7,20.1,19.2,22.0,98.6,0.2,0.7,0.4,212.8,2.6,1003,0,0.0,22.5,22.9,No Comment
2021-10-30 07:14:00,18.3,0.0,18.3,20.7,19.9,22.0,98.5,0.1,0.7,0.3,221.4,0.4,1003,0,0.0,22.5,22.9,No Comment
2021-10-30 07:15:00,18.9,0.0,18.9,21.4,20.5,22.0,98.6,0.2,0.7,0.5,189.1,6.1,1003,0,0.0,22.6,22.9,No Comment
2021-10-30 07:16:00,19.5,0.0,19.5,22.0,21.1,22.0,98.6,0.4,1.4,0.6,199.4,5.8,1003,0,0.0,22.6,22.9,No Comment
2021-10-30 07:17:00,20.1,0.0,20.1,22.7,21.8,22.0,98.5,0.1,1.1,0.3,192.0,1.0,1003,0,0.0,22.6,22.9,No Comment



**Cleaned Data Shape:** (238872, 18)


# **4. Post-Cleaning Data Profiling**

After cleaning, we re-profile the data to verify the effectiveness of the cleaning process. This includes updated summary statistics and a missing value report.# **4. Post-Cleaning Data Profiling**

After cleaning, we re-profile the data to verify the effectiveness of the cleaning process. This includes updated summary statistics and a missing value report.

In [5]:
# --- 4. Post-Cleaning Data Profiling ---
print(f"\n--- Post-Cleaning Data Profiling for {COUNTRY} ---")
print("\n**Summary Statistics (Cleaned Data):**")
display(dp.get_summary_statistics(df_cleaned.select_dtypes(include=['number'])))

print("\n**Missing Value Report (Cleaned Data):**")
dp.print_missing_value_report(df_cleaned, threshold=0)  # Check if any nulls remain


--- Post-Cleaning Data Profiling for Sierraleone ---

**Summary Statistics (Cleaned Data):**

--- Summary Statistics ---


,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB
count,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000,238872.000000
mean,445.471047,256.390219,251.600694,449.875422,431.288071,29.144408,68.701027,1.568610,2.297283,0.473658,177.697873,10.755592,999.467556,0.002060,0.001615,42.787084,42.607726
std,294.381895,263.173538,141.632581,300.153437,288.670600,4.202268,22.136594,1.225995,1.531143,0.265979,105.684921,8.054122,2.354675,0.045337,0.030674,11.545187,10.983106
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.400000,9.900000,0.000000,0.000000,0.000000,0.000000,0.000000,993.000000,0.000000,0.000000,11.700000,12.000000
25%,192.200000,3.100000,143.900000,188.300000,180.100000,25.900000,52.900000,0.400000,1.400000,0.400000,76.900000,5.200000,998.000000,0.000000,0.000000,33.800000,34.000000
50%,406.100000,166.300000,242.400000,403.900000,387.800000,29.300000,72.900000,1.500000,2.400000,0.500000,216.100000,10.600000,1000.000000,0.000000,0.000000,42.700000,43.100000
75%,687.400000,485.600000,348.900000,703.800000,671.800000,32.300000,85.600000,2.400000,3.400000,0.600000,254.700000,15.200000,1001.000000,0.000000,0.000000,52.200000,51.800000
max,1499.000000,946.000000,892.000000,1507.000000,1473.000000,39.900000,100.000000,19.200000,23.900000,4.100000,360.000000,98.400000,1006.000000,1.000000,2.400000,72.800000,70.400000



**Missing Value Report (Cleaned Data):**

--- Missing Value Report ---
No missing values found in the DataFrame.


# **5. Time Series Analysis**

We analyze the cleaned data over time, focusing on key variables such as GHI, DNI, DHI, and Tamb. This includes plotting time series and average daily/monthly profiles.

In [6]:
# --- 5. Time Series Analysis ---
print(f"\n--- Time Series Analysis for {COUNTRY} ---")

# GHI, DNI, DHI over Time
ep.plot_time_series(df_cleaned, ['GHI', 'DNI', 'DHI'], COUNTRY, title_suffix="Irradiance")

# Tamb over Time
ep.plot_time_series(df_cleaned, 'Tamb', COUNTRY, title_suffix="Ambient Temperature")

# Average Daily Profiles (hourly)
ep.plot_daily_average_profile(df_cleaned, ['GHI', 'Tamb'], COUNTRY)

# Average Monthly Profiles
ep.plot_monthly_average_profile(df_cleaned, ['GHI', 'Tamb'], COUNTRY)


--- Time Series Analysis for Sierraleone ---
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ghi_time_series.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_dni_time_series.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_dhi_time_series.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_tamb_time_series.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ghi_daily_profile.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_tamb_daily_profile.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ghi_monthly_profile.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_tamb_monthly_profile.png


### **6. Cleaning Impact Analysis**

In [7]:
print(f"\n--- Cleaning Impact Analysis for {COUNTRY} ---")
ep.plot_cleaning_impact(df_cleaned, 'ModA', 'ModB', COUNTRY)


--- Cleaning Impact Analysis for Sierraleone ---
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_cleaning_impact.png


### **7. Correlation & Relationship Analysis**

In [8]:
print(f"\n--- Correlation & Relationship Analysis for {COUNTRY} ---")

# Heatmap
ep.plot_correlation_heatmap(df_cleaned, ['GHI', 'DNI', 'DHI', 'TModA', 'TModB', 'Tamb', 'RH', 'WS', 'Precipitation'], COUNTRY)

# Scatter plots
ep.plot_scatter(df_cleaned, 'WS', 'GHI', COUNTRY, title_suffix="(Wind Speed vs GHI)")
ep.plot_scatter(df_cleaned, 'WSgust', 'GHI', COUNTRY, title_suffix="(Wind Gust vs GHI)")
ep.plot_scatter(df_cleaned, 'RH', 'Tamb', COUNTRY, title_suffix="(Relative Humidity vs Ambient Temperature)")
ep.plot_scatter(df_cleaned, 'RH', 'GHI', COUNTRY, title_suffix="(Relative Humidity vs GHI)")


--- Correlation & Relationship Analysis for Sierraleone ---
  Saved matplotlib plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_correlation_heatmap.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ghi_vs_ws_scatter.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ghi_vs_wsgust_scatter.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_tamb_vs_rh_scatter.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ghi_vs_rh_scatter.png


### **8. Wind & Distribution Analysis**

In [9]:
print(f"\n--- Wind & Distribution Analysis for {COUNTRY} ---")

# Wind Rose
ep.plot_wind_rose(df_cleaned, 'WS', 'WD', COUNTRY)

# Histograms
ep.plot_distribution(df_cleaned, 'GHI', COUNTRY, hist_type='histogram')
ep.plot_distribution(df_cleaned, 'WS', COUNTRY, hist_type='histogram')
ep.plot_distribution(df_cleaned, 'RH', COUNTRY, hist_type='histogram')


--- Wind & Distribution Analysis for Sierraleone ---


c:\Users\Perserverence\Documents\Python_Scripts\solar-challenge-week1\notebooks\..\scripts\eda_plots.py:233: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_wind_rose.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ghi_distribution.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ws_distribution.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_rh_distribution.png


### **9. Bubble Chart**

In [10]:
print(f"\n--- Bubble Chart for {COUNTRY} ---")

# GHI vs. Tamb with bubble size = RH
ep.plot_bubble_chart(df_cleaned, 'Tamb', 'GHI', 'RH', COUNTRY)

# GHI vs. Tamb with bubble size = BP (if BP is relevant as a bubble size)
ep.plot_bubble_chart(df_cleaned, 'Tamb', 'GHI', 'BP', COUNTRY)


--- Bubble Chart for Sierraleone ---
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ghi_vs_tamb_rh_bubble.png
  Saved plotly plot: c:\Users\Perserverence\Documents\Python_Scripts\reports\figures\sierraleone_ghi_vs_tamb_bp_bubble.png
